# Multiclass SVM 구현

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

C:\ana3\envs\t_f2.2\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ana3\envs\t_f2.2\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\ana3\envs\t_f2.2\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [2]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [3]:
def standardization(train, test):
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    return train, test

X_train, X_test = standardization(X_train, X_test)

### one vs rest svm

In [7]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [31]:
from sklearn.svm import SVC

C = [10**(-5), 10, 10**(5)]
tmp = []

for i in range(len(y_train.columns)):
    svm = SVC(kernel='rbf', C = 10, gamma='auto')
    svm.fit(X_train,y_train.iloc[:,i])
    tmp.append(svm)

In [32]:
tmp

[SVC(C=10, gamma='auto'), SVC(C=10, gamma='auto'), SVC(C=10, gamma='auto')]

In [17]:
X_test

array([[-0.14887164, -0.374155  ,  0.26676379,  0.13941922],
       [ 0.31817664, -0.60559109,  0.54756778,  0.00972692],
       [ 0.31817664, -1.06846325,  1.05301496,  0.26911151],
       [-1.5500165 , -1.7627715 , -1.36189934, -1.15750374],
       [ 0.0846525 ,  0.32015325,  0.60372857,  0.7878807 ],
       [ 0.78522493, -0.14271892,  0.99685416,  0.7878807 ],
       [-0.84944407,  1.70876975, -1.24957775, -1.15750374],
       [ 0.20141457, -0.14271892,  0.60372857,  0.7878807 ],
       [-0.38239578,  2.63451409, -1.30573855, -1.28719604],
       [-0.38239578, -1.29989934,  0.15444219,  0.13941922],
       [ 0.66846286,  0.08871717,  0.99685416,  0.7878807 ],
       [-0.38239578,  1.0144615 , -1.36189934, -1.28719604],
       [-0.49915786,  0.78302542, -1.13725615, -1.28719604],
       [ 0.43493872, -0.60559109,  0.60372857,  0.7878807 ],
       [ 0.55170079, -1.7627715 ,  0.37908538,  0.13941922],
       [ 0.55170079,  0.55158933,  0.54756778,  0.52849611],
       [-1.19973028,  0.

In [33]:
shape_zero = np.zeros_like(X_test)

In [63]:
y_pred = pd.DataFrame(columns=['setosa','versicolor','virginica'])
for i in range(shape_zero.shape[0]):
    setosa = tmp[0].decision_function(X_test)[i]
    versicolor = tmp[1].decision_function(X_test)[i]
    virginica = tmp[2].decision_function(X_test)[i]
    decision = np.argmax([setosa,versicolor,virginica])
    if decision == 0:
        #dic = {'setosa':1,'versicolor':0,'virginica':0}
        y_pred.at[i,'setosa'] = 1
    elif decision ==1:
        #dic = {'setosa':0,'versicolor':1,'virginica':0}
        y_pred.at[i,'versicolor'] =1
    elif decision ==2:
        #dic = {'setosa':0,'versicolor':0,'virginica':1}
        y_pred.at[i,'virginica'] = 1
    #y_pred.append(dic, ignore_index=True)
y_pred = y_pred.fillna(0)
y_pred

,setosa,versicolor,virginica
0,0,1,0
1,0,1,0
2,0,0,1
3,0,1,0
4,0,0,1
5,0,0,1
6,1,0,0
7,0,0,1
8,1,0,0
9,0,1,0


In [65]:
from sklearn.metrics import accuracy_score

print('accuracy score:',accuracy_score(y_test,y_pred))

accuracy score: 0.9


### Compare with scikit-learn

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

X_train, X_test = standardization(X_train, X_test)

In [70]:
svc_clf = SVC(kernel='rbf', C = 10, gamma='auto')
svc_clf.fit(X_train,y_train)
svc_clf_pred = svc_clf.predict(X_test)
print('accuracy score:',accuracy_score(y_test,svc_clf_pred))

accuracy score: 0.9


### result: one vs rest와 라이브러리에서 제공된 accuracy_score 결과값 같다.